This application turns any word or phrase into a mini language lesson that fits your level (A1–C2). It gives clear translations, simple explanations, and example sentences, plus extra grammar help for German verbs and nouns.
The example sentences are converted to audio format using the OpenAI Text-to-Speech API.

In [ ]:
import os
import json
from dotenv import load_dotenv
from pathlib import Path
from openai import OpenAI

load_dotenv(override=True)

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GROQ_API_URL = "https://api.groq.com/openai/v1"
GROQ_MODEL = "openai/gpt-oss-120b"

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_TTS_MODEL = "gpt-4o-mini-tts"

In [ ]:
def create_learning_card_prompt(native_lang,expression,target_lang,level):
    json_template = {
        "original": "expression_with_article",
        "translation": "translated_expression",
        "explanation": "...",
        "examples": [
            {
                "sentence": "sentence",
                "translation": "translated_sentence"
            }
            # ... more examples
        ],
        "noun": {
            "plural": "plural_noun_with_article",
            "example": "sentence",
            "translation": "translated_sentence"
        },
        "verb": {
            "Präsens": "Er <pres_3sg>",
            "Präteritum": "Sie <past_3sg>",
            "Perfekt": "Es <auxiliary> <past_participle>"
        }
    }

    system_prompt = f"""
    You are a lang tutor. Vars: native="{native_lang}", expr="{expression}", target="{target_lang}", level="{level}".

    Translate expr bidirectionally (native→target or vice versa; fix typos; typical usage for level). Capitalize translation.

    If expr in target: explain meaning in native (real-world, 1-3 sentence).

    Write 3 examples: target sentences at {level} (A1=simple,C2=advanced), each + native translation.

    If target=German & noun: add plural + 1 plural example w/ translation.
    If target=German & verb: add conj: 3sg pres · 3sg past · aux+part (haben/sein).

    Respond in JSON: {json_template}
    """

    user_prompt = f'Translate the following expression using the provided input data: "{expression}"'

    return system_prompt, user_prompt

In [ ]:
system_prompt, user_prompt = create_learning_card_prompt("Ukrainian", "fahren", "German", "B1")
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

groq = OpenAI(base_url=GROQ_API_URL, api_key=GROQ_API_KEY)
response = groq.chat.completions.create(
    model=GROQ_MODEL,
    messages=messages,
    response_format={"type": "json_object"})

result = json.loads(response.choices[0].message.content)
print(json.dumps(result, indent="\t", ensure_ascii=False))

# Transform example sentences into audio files
sentences = [example["sentence"] for example in result["examples"]]

openai = OpenAI(api_key=OPENAI_API_KEY)
for i, sentence in enumerate(sentences):
    response = openai.audio.speech.create(
        model=OPENAI_TTS_MODEL,
        voice="fable",
        input=sentence
    )

    filename = Path(f"sentence{i}.mp3")
    response.write_to_file(filename)

    print(f"Saved sentence #{i} to {filename}")